In [21]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from geopy import distance
import geopandas
import math

In [22]:
token = open(".mapbox_token").read()    # API token for mapbox.

In [23]:
"""
Reading data from csv file, clean the data and return a dataframe
"""
def read_csv(file_path):
    # read data
    df = pd.read_csv(file_path, header=None, encoding='unicode_escape', on_bad_lines="skip")
    df.columns = ["R_Time", "Comm_ID", "Pkt_Set", "Pkt_ID", "S_Lat", "S_Lng", "Pkt_BW", "Pkt_CR", "Pkt_Freq", "Pkt_SF", "Pkt_Tx_Pwr", "Pkt_RSSI", "Now_RSSI", "Pkt_SNR", "R_Lat", "R_Lng"]
    
    # data cleaning
    # drop rows with non-numeric latitude & longitudes
    df['S_Lat'] = pd.to_numeric(df['S_Lat'], errors='coerce')
    df['S_Lng'] = pd.to_numeric(df['S_Lng'], errors='coerce')
    df.dropna(inplace=True)
    # drop rows with S_Lat out of range
    df = df[df['S_Lat'] < 90]
    # delete row if Pkt_SF is not 12.0
    df = df[df['Pkt_SF'] == 12.0]
    
    return df

In [24]:
# read locations of car parks into dataframe
carpark_df = pd.read_csv("./data/carpark_locs.txt", header=None)
carpark_df.columns = ["id", "lat", "lng"]

In [25]:
# read all data into dataframes
p1 = read_csv("./data/P1.csv")
p3 = read_csv("./data/P3.csv")
p8 = read_csv("./data/P8.csv")
p18 = read_csv("./data/P18.csv")
p26 = read_csv("./data/P26.csv")
p31 = read_csv("./data/P31.csv")
p33 = read_csv("./data/P33.csv")
p43 = read_csv("./data/P43.csv")

In [26]:
def save_html(df, id):
    center = {"lat": -31.980256, "lon": 115.817928}
    fig = px.scatter_mapbox(df, lat="R_Lat", lon="R_Lng", color="Pkt_SNR",
                    color_continuous_scale=px.colors.sequential.Plasma, zoom=14,
                    center=center,
                    width=800, height=600, opacity=0.5)

    fig.add_trace(go.Scattermapbox(
            lat=carpark_df["lat"],
            lon=carpark_df["lng"],
            mode='markers+text',
            marker= {'size': 20, 'color':'#39b4ac'},
            text=carpark_df["id"],
            textposition="middle center"
        ))

    fig.update_layout(
        mapbox = {
            'accesstoken': token,
            'style': "basic"},
        showlegend = False)

    fig.show()
    fig.write_html(f"./htmls/{id}.html")

In [27]:
# save htmls for all car park data
dfs = [p1, p3, p8, p18, p26, p31, p33, p43]
ids = ["p1", "p3", "p8", "p18", "p26", "p31", "p33", "p43"]
for i in range(len(dfs)):
    save_html(dfs[i], ids[i])

In [28]:
# save carpark_df values into dictionary
carpark_dict = {}
for index, row in carpark_df.iterrows():
    carpark_dict[row["id"].lower()] = (row["lat"], row["lng"])

In [29]:
# calculate distance from data collected to the center (carpark coordinates)
def calculate_distance(df, center):
    df["distance"] = df.apply(lambda row: distance.distance((row["R_Lat"], row["R_Lng"]), carpark_dict[center]).meters, axis=1)
    return df

In [30]:
# calculate distance for all data
for i in range(len(dfs)):
    dfs[i] = calculate_distance(dfs[i], ids[i])

full_df_distance = pd.concat(dfs)

In [31]:
# plot distance vs SNR
def plot_distance_snr(df):
    # change size of plot
    fig = px.scatter(df, x="distance", y="Pkt_SNR", color="Pkt_SNR", color_continuous_scale=px.colors.sequential.Plasma)
    # add x axis title
    fig.update_xaxes(title_text='Distance (m)')

    fig.add_hline(y=0, line_width=3, line_color="green", opacity=0.7)
    fig.add_annotation(x=800, y=0, text="0 dBm", showarrow=False, yshift=10, font=dict(color="green", size=15))

    fig.add_hline(y=-20, line_width=3, line_color="#39b4ac", opacity=0.7)
    fig.add_annotation(x=800, y=-20, text="-20 dBm", showarrow=False, yshift=10, font=dict(color="#39b4ac", size=15))

    fig.update_layout(width=1000, height=500)
    
    fig.show()
    fig.write_html("./htmls/distance_snr.html")

# plot distance vs RSSI
def plot_distance_rssi(df):
    fig = px.scatter(df, x="distance", y="Pkt_RSSI", color="Pkt_RSSI", color_continuous_scale=px.colors.sequential.ice)
    fig.update_xaxes(title_text='Distance (m)')
    fig.update_layout(width=1000, height=500)
    fig.show()
    fig.write_html("./htmls/distance_rssi.html")


In [32]:
plot_distance_snr(full_df_distance)

In [33]:
plot_distance_rssi(full_df_distance)

In [34]:
percentage_snr_below_20 = len(full_df_distance[full_df_distance["Pkt_SNR"] < -20])/len(full_df_distance)
print(f"Percentage of SNR below -20 dB: {percentage_snr_below_20*100}%")

Percentage of SNR below -20 dB: 0.9050943884147918%


In [35]:
# filter out rows with SNR < -20 because they can't be demodulated correctly
snr_20_df = full_df_distance.copy()
snr_20_df = snr_20_df[snr_20_df["Pkt_SNR"] >= -20]
# remove some unused columns
columns_to_remove = ['R_Time', 'Comm_ID', 'Pkt_Set', 'Pkt_ID','Pkt_BW', 'Pkt_CR', 'Pkt_Freq', 'Pkt_SF', 'Pkt_Tx_Pwr']
snr_20_df.drop(columns=columns_to_remove, inplace=True)

In [36]:
snr_20_df.describe()

,S_Lat,S_Lng,Pkt_RSSI,Now_RSSI,Pkt_SNR,R_Lat,R_Lng,distance
count,3832.000000,3832.000000,3832.000000,3832.000000,3832.000000,3832.000000,3832.000000,3832.000000
mean,-31.980700,115.818218,-91.408925,-120.218163,2.029293,-31.979722,115.818340,172.852540
std,0.098560,0.016517,19.755450,5.642800,9.758824,0.004477,0.002536,164.034052
min,-37.981158,115.120238,-116.000000,-129.000000,-20.000000,-31.987751,115.812746,0.756058
25%,-31.983583,115.817538,-109.000000,-123.000000,-6.000000,-31.984227,115.816272,40.586087
50%,-31.978955,115.818801,-97.000000,-122.000000,6.000000,-31.978989,115.818680,110.946575
75%,-31.976212,115.820756,-77.000000,-122.000000,10.500000,-31.976188,115.820216,284.194326
max,-31.176284,115.889810,-44.000000,-82.000000,13.750000,-31.970764,115.824989,817.175551


In [37]:
snr_0_df = snr_20_df.copy()
snr_0_df = snr_0_df[snr_0_df["Pkt_SNR"] >= 0]
snr_0_df.describe()

,S_Lat,S_Lng,Pkt_RSSI,Now_RSSI,Pkt_SNR,R_Lat,R_Lng,distance
count,2459.000000,2459.000000,2459.000000,2459.000000,2459.000000,2459.000000,2459.000000,2459.000000
mean,-31.981429,115.818334,-81.078081,-120.204961,8.588756,-31.979430,115.818351,80.765630
std,0.121660,0.008671,17.475416,5.770238,3.350814,0.004336,0.002655,79.743053
min,-37.981158,115.419778,-110.000000,-129.000000,0.000000,-31.987183,115.812746,0.756058
25%,-31.983550,115.816344,-96.000000,-123.000000,6.375000,-31.983393,115.816295,28.624177
50%,-31.978258,115.818741,-82.000000,-122.000000,9.750000,-31.978191,115.818724,52.031721
75%,-31.976140,115.819895,-70.000000,-122.000000,11.250000,-31.976160,115.820273,109.154510
max,-31.576217,115.883054,-44.000000,-82.000000,13.750000,-31.972489,115.824989,557.049033


In [44]:
def get_circle_coords(radius_km):
    circle_coordinates = {}
    num_points=1000
    # for every lat lon in carpark_df, generate a circle with radius
    for _, row in carpark_df.iterrows():
        key = row["id"]
        center_lat = row["lat"]
        center_lon = row["lng"]
        circle_coordinates[key] = []
        for bearing in range(0, 360, 1):  # Generate coordinates every 1 degree
            # Calculate the point on the circle
            point = distance.distance(kilometers=radius_km).destination((center_lat, center_lon), bearing)
            circle_coordinates[key].append((point.latitude, point.longitude))
    return circle_coordinates

In [45]:
# get 75% percentile for distance from snr_20_df & snr_0_df
distance_75_snr_20 = snr_20_df["distance"].quantile(0.75)/1000      # divide by 1000 to convert to km
distance_75_snr_0 = snr_0_df["distance"].quantile(0.75)/1000
cc_0 = get_circle_coords(distance_75_snr_0)   # circle coordinates for SNR>0
cc_20 = get_circle_coords(distance_75_snr_20)  # circle coordinates for SNR>-20

In [46]:
center = {"lat": -31.980256, "lon": 115.817928}     # coordinates for UWA campus center

fig = go.Figure(
    data=go.Scattermapbox(
        name="Carpark",
        lat=carpark_df["lat"],
        lon=carpark_df["lng"],
        mode='markers+text',
        marker= {'size': 20, 'color':'#39b4ac'},
        text=carpark_df["id"],
        textposition="middle center"
    )
)

for key in cc_0:
    fig.add_trace(go.Scattermapbox(
        lat=[x[0] for x in cc_0[key]],
        lon=[x[1] for x in cc_0[key]],
        mode='lines',
        line=dict(width=1, color='#f55500'),
        fill='toself',
        fillcolor='rgba(245, 85, 0, 0.1)',
        hoverinfo="none"
    ))

for key in cc_20:
    fig.add_trace(go.Scattermapbox(
        lat=[x[0] for x in cc_20[key]],
        lon=[x[1] for x in cc_20[key]],
        mode='lines',
        line=dict(width=1, color='#f7a000'),
        fill='toself',
        fillcolor='rgba(247, 160, 0, 0.1)',
        hoverinfo="none"
    ))


fig.update_layout(
    width=1000, height=800,
    mapbox = {
        'accesstoken': token,
        'style': "basic",
        'center': go.layout.mapbox.Center(lat=center["lat"], lon=center["lon"]),
        'zoom': 14},
        showlegend=False)

fig.show()
fig.write_html("./htmls/coverage.html")